In [20]:
import sys
import regex
import re
import re
def revers(seq):
    codes={"A":"A", "C":"C", "G":"G", "T":"T", 
           "R":"[AG]", "S":"[GC]", "B":"[CGT]", "Y":"[CT]", 
           "W":"[AT]", "D":"[AGT]", "K":"[GT]", "N":"[ACGT]", 
           "H":"[ACT]", "M":"[AC]", "V":"[ACG]", "X": "[ACGT]"}
    
    d={'A':'T','G':'C','C':'G','T':'A',
      'R':'Y','S':'S','B':'V','Y':'R',
       'W':'W','D':'H','K':'M','N':'N',
       'H':'D','M':'K','V':'B','X':'X'
      }
    new=''
    for i in seq[::-1]:
        new+=d[i]
    return new

def replace_ambiguity_codes(sequence):
    codes={"A":"A", "C":"C", "G":"G", "T":"T", 
           "R":"[AG]", "S":"[GC]","B":"[CGT]", "Y":"[CT]", "W":"[AT]",
           "D":"[AGT]", "K":"[GT]", "N":"[ACGT]", "H":"[ACT]", "M":"[AC]", "V":"[ACG]", "X": "[ACGT]"}
    regexlist=[]
    for base in sequence:
        if not base in codes:
            print("Unidentified nucleotide code in primer:", base)
            sys.exit()
        else:
            regexlist.append(codes[base])

    return ''.join(regexlist)


'''
In solico PCR
def get_target_region(input_fasta,f_primer,r_primer):
    f_primer=replace_ambiguity_codes(f_primer)
    r_primer=replace_ambiguity_codes(r_primer)
    f=open(input_fasta,'r')
    fasta_data=f.read().replace('\r','').split('>')[1:]
    f.close()
    d1={} # d1: key: ID  value: ATCG
    target={}
    for each_seq in fasta_data:
        d1[each_seq.split(' ')[0]]=each_seq.split('\n',1)[1].replace('\n','')
    for each_seq in d1:
        findall = re.findall(f_primer, d1[each_seq])
        if len(findall) == 1:
            start=d1[each_seq].split(findall[0])[1]
            findall2 = re.findall(r_primer,start)
            if len(findall2)==1:
                target[each_seq]=start.split(findall2[0])[0]
    return len(target),len(d1),len(target)/len(d1),target
'''




def cov(f,r,d):
    result=[]
    for i in d:
        hit=[]
        
        for j in d[i]:
            forward=re.findall(replace_ambiguity_codes(f),j)
            if len(forward) ==1:
                reverse=re.findall(replace_ambiguity_codes(revers(r)),j.split(forward[0])[1])
                if len(reverse) ==1:
                    hit.append(j)
        result.append(str((len(hit)/len(d[i]))*100)+'%')
        #print(str((len(hit)/len(d[i]))*100)+'%')
    return result
        #print(len(d[i]))
        #print(i)
        
def cov_e1(f,r,d):
    result=[]
    for i in d:
        hit=[]
        
        for j in d[i]:
            forward=regex.findall('('+replace_ambiguity_codes(f)+'){e<=1}',j)
            if len(forward) ==1:
                reverse=regex.findall('('+replace_ambiguity_codes(revers(r))+'){e<=1}',j.split(forward[0])[1])
                if len(reverse) ==1:
                    hit.append(j)
        result.append(str((len(hit)/len(d[i]))*100)+'%')
        #print(str((len(hit)/len(d[i]))*100)+'%')
    return result
        #print(len(d[i]))
        #print(i)                    

# This script use three database to calculate different primer coverge with zero mismatch and one mismatch

v1-v3   AGAGTTTGATCCTGGCTCAG    ATTACCGCGGCTGCTGG       27F 534R

v4:    GTGYCAGCMGCCGCGGTAA     GGACTACNVGGGTWTCTAAT    515F (Parada)–806R (Apprill)

v3-v4   CCTACGGGNGGCWGCAG      GGACTACNVGGGTWTCTAAT    341F 806R

v3-v4   CCTACGGGNGGCWGCAG      GACTACHVGGGTATCTAATCC   341F  785R

v4-v5   GTGYCAGCMGCCGCGGTAA    CCGYCAATTYMTTTRAGTTT    515F  926R

v3-v4  YCTACGGRNGGCHGCAR        RGACTAMNVRGGTMTCTAAT   341F  806R CPR

v3-v4  YCTACGGRNGGCHGCAR        RGACTAMNVRGGTHTCTAAT   341F  806R CPR2

## MiDAS4

In [21]:
d={}
with open(r'./data/midas4.fasta') as f:
    data=f.read().split('>')[1:]
    for i in data:
        if 'Patesci' in i:
            if i.split('\n')[0].split(',')[2] not in d:
                d[i.split('\n')[0].split(',')[2]] = [i.split('\n',1)[1].replace('\n','')]
            else:
                d[i.split('\n')[0].split(',')[2]].append(i.split('\n',1)[1].replace('\n',''))
# merge denovo midas class
dd={}
dd['denovo_midas_class']=[]
for i in d:
    if 'midas' in i:
        for j in d[i]:
            dd['denovo_midas_class'].append(j)
    else:
        dd[i] = d[i]
midas=dd

a1=cov('AGAGTTTGATCCTGGCTCAG','ATTACCGCGGCTGCTGG',midas) #27F 534R 
a2=cov('GTGYCAGCMGCCGCGGTAA','GGACTACNVGGGTWTCTAAT',midas) #515F (Parada)–806R (Apprill)
a4=cov('CCTACGGGNGGCWGCAG','GGACTACNVGGGTWTCTAAT',midas) #341F 806R
a3=cov('CCTACGGGNGGCWGCAG','GACTACHVGGGTATCTAATCC',midas) #341F  785R
a5=cov('GTGYCAGCMGCCGCGGTAA','CCGYCAATTYMTTTRAGTTT',midas) #515F  926R
a6=cov('YCTACGGRNGGCHGCAR','RGACTAMNVRGGTMTCTAAT',midas) #CPR v1
a7=cov('YCTACGGRNGGCHGCAR','RGACTAMNVRGGTHTCTAAT',midas) #CPR v2

b1=cov_e1('AGAGTTTGATCCTGGCTCAG','ATTACCGCGGCTGCTGG',midas) #27F 534R 
b2=cov_e1('GTGYCAGCMGCCGCGGTAA','GGACTACNVGGGTWTCTAAT',midas) #515F (Prd)–806R (pprill)
b4=cov_e1('CCTACGGGNGGCWGCAG','GGACTACNVGGGTWTCTAAT',midas) #341F 806R
b3=cov_e1('CCTACGGGNGGCWGCAG','GACTACHVGGGTATCTAATCC',midas) #341F  785R
b5=cov_e1('GTGYCAGCMGCCGCGGTAA','CCGYCAATTYMTTTRAGTTT',midas) #515F  926R
b6=cov_e1('YCTACGGRNGGCHGCAR','RGACTAMNVRGGTMTCTAAT',midas) #CPR v1
b7=cov_e1('YCTACGGRNGGCHGCAR','RGACTAMNVRGGTHTCTAAT',midas) #CPR v2

l=[len(midas[i]) for i in midas]
m= {'Taxonomy':list(midas.keys()),
    'No.Seq':l,
    'V1-V3(27F-534R)': a1, 
    'V4(515F-806R)': a2,
    'V3-V4(341F-785R)': a3, 
    'V3-V4(341F-806R)': a4,
    'V4-V5(515F-926R)': a5, 
    'V3-V4(CPRv1)': a6,
       'V3-V4(CPRv2)': a7
   }

m1= {'Taxonomy':list(midas.keys()),
    'No.Seq':l,
    'V1-V3(27F-534R)': b1, 
    'V4(515F-806R)': b2,
    'V3-V4(341F-785R)': b3, 
    'V3-V4(341F-806R)': b4,
    'V4-V5(515F-926R)': b5, 
    'V3-V4(CPRv1)': b6,
       'V3-V4(CPRv2)': b7
   }

import pandas as pd
f1=pd.DataFrame(data=m)
f1.sort_values("No.Seq",inplace=True,ascending=False)
f1.to_csv('./result/midas.csv',index=False)

f11=pd.DataFrame(data=m1)
f11.sort_values("No.Seq",inplace=True,ascending=False)
f11.to_csv('./result/midas_one_mismatch.csv',index=False)

# Silva


In [22]:
silva={}
with open(r'./data/silva_CPR.fasta') as f:
    data=f.read().split('>')[1:]
    for i in data:
        if 'Patesci' in i:
            if i.split(';')[2] not in silva:
                silva[i.split(';')[2]] = [i.split('\n',1)[1].replace('\n','').replace('U','T')]
            else:
                silva[i.split(';')[2]].append(i.split('\n',1)[1].replace('\n','').replace('U','T'))

                
a1=cov('AGAGTTTGATCCTGGCTCAG','ATTACCGCGGCTGCTGG',silva) #27F 534R 
a2=cov('GTGYCAGCMGCCGCGGTAA','GGACTACNVGGGTWTCTAAT',silva) #515F (Parada)–806R (Apprill)
a4=cov('CCTACGGGNGGCWGCAG','GGACTACNVGGGTWTCTAAT',silva) #341F 806R
a3=cov('CCTACGGGNGGCWGCAG','GACTACHVGGGTATCTAATCC',silva) #341F  785R
a5=cov('GTGYCAGCMGCCGCGGTAA','CCGYCAATTYMTTTRAGTTT',silva) #515F  926R
a6=cov('YCTACGGRNGGCHGCAR','RGACTAMNVRGGTMTCTAAT',silva) #CPR v1
a7=cov('YCTACGGRNGGCHGCAR','RGACTAMNVRGGTHTCTAAT',silva) #CPR v2

b1=cov_e1('AGAGTTTGATCCTGGCTCAG','ATTACCGCGGCTGCTGG',silva) #27F 534R 
b2=cov_e1('GTGYCAGCMGCCGCGGTAA','GGACTACNVGGGTWTCTAAT',silva) #515F (Prd)–806R (pprill)
b4=cov_e1('CCTACGGGNGGCWGCAG','GGACTACNVGGGTWTCTAAT',silva) #341F 806R
b3=cov_e1('CCTACGGGNGGCWGCAG','GACTACHVGGGTATCTAATCC',silva) #341F  785R
b5=cov_e1('GTGYCAGCMGCCGCGGTAA','CCGYCAATTYMTTTRAGTTT',silva) #515F  926R
b6=cov_e1('YCTACGGRNGGCHGCAR','RGACTAMNVRGGTMTCTAAT',silva) #CPR v1
b7=cov_e1('YCTACGGRNGGCHGCAR','RGACTAMNVRGGTHTCTAAT',silva) #CPR v2

l=[len(silva[i]) for i in silva]
m= {'Taxonomy':list(silva.keys()),
    'No.Seq':l,
    'V1-V3(27F-534R)': a1, 
    'V4(515F-806R)': a2,
    'V3-V4(341F-785R)': a3, 
    'V3-V4(341F-806R)': a4,
    'V4-V5(515F-926R)': a5, 
    'V3-V4(CPRv1)': a6,
       'V3-V4(CPRv2)': a7
   }
m1= {'Taxonomy':list(silva.keys()),
    'No.Seq':l,
    'V1-V3(27F-534R)': b1, 
    'V4(515F-806R)': b2,
    'V3-V4(341F-785R)': b3, 
    'V3-V4(341F-806R)': b4,
    'V4-V5(515F-926R)': b5, 
    'V3-V4(CPRv1)': b6,
       'V3-V4(CPRv2)': b7
   }
import pandas as pd
f2=pd.DataFrame(data=m)
f2.sort_values("No.Seq",inplace=True,ascending=False)
f2.to_csv('./result/silva.csv',index=False)

f22=pd.DataFrame(data=m1)
f22.sort_values("No.Seq",inplace=True,ascending=False)
f22.to_csv('./result/silva_one_mismatch.csv',index=False)

# CPR DB

In [23]:
# get fasta:

d={}
with open(r'./data/CPR_DB_16S_withtax.fasta') as f:
    data=f.read().split('>')[1:]
    for i in data:
        if i.split('\n')[0].split(';')[2] not in d:
            d[i.split(';')[2]] = [i.split('\n',1)[1].replace('\n','')]
        else:
            d[i.split(';')[2]].append(i.split('\n',1)[1].replace('\n',''))
            
cpr=d


a1=cov('AGAGTTTGATCCTGGCTCAG','ATTACCGCGGCTGCTGG',cpr) #27F 534R 
a2=cov('GTGYCAGCMGCCGCGGTAA','GGACTACNVGGGTWTCTAAT',cpr) #515F (Parada)–806R (Apprill)
a4=cov('CCTACGGGNGGCWGCAG','GGACTACNVGGGTWTCTAAT',cpr) #341F 806R
a3=cov('CCTACGGGNGGCWGCAG','GACTACHVGGGTATCTAATCC',cpr) #341F  785R
a5=cov('GTGYCAGCMGCCGCGGTAA','CCGYCAATTYMTTTRAGTTT',cpr) #515F  926R
a6=cov('YCTACGGRNGGCHGCAR','RGACTAMNVRGGTMTCTAAT',cpr) #CPR v1
a7=cov('YCTACGGRNGGCHGCAR','RGACTAMNVRGGTHTCTAAT',cpr) #CPR v2

b1=cov_e1('AGAGTTTGATCCTGGCTCAG','ATTACCGCGGCTGCTGG',cpr) #27F 534R 
b2=cov_e1('GTGYCAGCMGCCGCGGTAA','GGACTACNVGGGTWTCTAAT',cpr) #515F (Prd)–806R (pprill)
b4=cov_e1('CCTACGGGNGGCWGCAG','GGACTACNVGGGTWTCTAAT',cpr) #341F 806R
b3=cov_e1('CCTACGGGNGGCWGCAG','GACTACHVGGGTATCTAATCC',cpr) #341F  785R
b5=cov_e1('GTGYCAGCMGCCGCGGTAA','CCGYCAATTYMTTTRAGTTT',cpr) #515F  926R
b6=cov_e1('YCTACGGRNGGCHGCAR','RGACTAMNVRGGTMTCTAAT',cpr) #CPR v1
b7=cov_e1('YCTACGGRNGGCHGCAR','RGACTAMNVRGGTHTCTAAT',cpr) #CPR v2


l=[len(cpr[i]) for i in cpr]
m= {'Taxonomy':list(cpr.keys()),
    'No.Seq':l,
    'V1-V3(27F-534R)': a1, 
    'V4(515F-806R)': a2,
    'V3-V4(341F-785R)': a3, 
    'V3-V4(341F-806R)': a4,
    'V4-V5(515F-926R)': a5, 
    'V3-V4(CPRv1)': a6,
       'V3-V4(CPRv2)': a7
   }
m1= {'Taxonomy':list(cpr.keys()),
    'No.Seq':l,
    'V1-V3(27F-534R)': b1, 
    'V4(515F-806R)': b2,
    'V3-V4(341F-785R)': b3, 
    'V3-V4(341F-806R)': b4,
    'V4-V5(515F-926R)': b5, 
    'V3-V4(CPRv1)': b6,
       'V3-V4(CPRv2)': b7
   }
import pandas as pd
f3=pd.DataFrame(data=m)
f3.sort_values("No.Seq",inplace=True,ascending=False)
f3.to_csv('./result/cpr.csv',index=False)

f33=pd.DataFrame(data=m1)
f33.sort_values("No.Seq",inplace=True,ascending=False)
f33.to_csv('./result/cpr_one_mismatch.csv',index=False)